In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder



HERE STARTS BERT

In [3]:
# from transformers import AutoTokenizer, AutoModel
import torch
# from sklearn.svm import SVC
from sentence_transformers import SentenceTransformer , util

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2", device="cuda")


In [5]:
df_bert = pd.read_csv(r'D:\Projects\ATS_Model\cs_resume_dataset_realistic.csv')

In [6]:
X_bert = df_bert['SKILLS'].tolist()
y_bert = df_bert['JOB_CATEGORY'].tolist()

In [7]:
X_embeddings = model.encode(X_bert, show_progress_bar=True, convert_to_tensor=False)

Batches:   0%|          | 0/46875 [00:00<?, ?it/s]

In [8]:
le = LabelEncoder()
y_bert_encoded = le.fit_transform(y_bert)


In [ ]:
# resume_vec = model.encode(X_bert,show_progress_bar=True,convert_to_tensor=True)

Batches: 100%|██████████| 313/313 [00:02<00:00, 134.93it/s]


In [ ]:
# job_vec = model.encode(y_bert, show_progress_bar=True, convert_to_tensor=True)

Batches: 100%|██████████| 313/313 [00:02<00:00, 152.11it/s]


In [ ]:
# X_embeddings = X_embeddings.cpu()


In [9]:
X_train_bert, X_test_bert, y_train_bert, y_test_bert = train_test_split(
    X_embeddings, y_bert_encoded, test_size=0.3, random_state=42 , stratify = y_bert
)

In [10]:
dtrain = xgb.DMatrix(X_train_bert, label=y_train_bert)
dtest = xgb.DMatrix(X_test_bert, label=y_test_bert)


In [11]:
params = {
    "objective": "multi:softprob",  # multi-class classification with probabilities
    "num_class": len(set(y_bert)),
    "eval_metric": "mlogloss",
    "tree_method": "gpu_hist",      # if you want GPU, else use 'hist'
    "gpu_id": 0,
    "max_depth": 4,
    "learning_rate": 0.1,
    "n_estimators": 300,
    'min_child_weight': 10,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'gamma': 0.5,
    "reg_alpha": 0.05,              # Higher L1 regularization
    "reg_lambda": 0.5,              # Higher L2 regularization
    "grow_policy": "depthwise"
}

In [12]:
num_round = 300
# bst = xgb.train(params, dtrain, num_round)
bst = xgb.train(
    params,
    dtrain,
    num_boost_round=num_round,
    evals=[(dtrain, "train"), (dtest, "test")],  # track both train and test
    verbose_eval=10,  # prints info every 10 rounds
    early_stopping_rounds=20
)

C:\Users\ishan\AppData\Roaming\Python\Python313\site-packages\xgboost\callback.py:386: UserWarning: [20:38:29] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  self.starting_round = model.num_boosted_rounds()
C:\Users\ishan\AppData\Roaming\Python\Python313\site-packages\xgboost\callback.py:386: UserWarning: [20:38:29] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  self.starting_round = model.num_boosted_rounds()
C:\Users\ishan\AppData\Roaming\Python\Python313\site-packages\xgboost\callback.py:386: UserWarning: [20:38:29] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "n_estimators" } are not used.

  self.starting_round = model.num_boosted_

[0]	train-mlogloss:2.02185	test-mlogloss:2.02214
[10]	train-mlogloss:0.69103	test-mlogloss:0.69201
[20]	train-mlogloss:0.31164	test-mlogloss:0.31293
[30]	train-mlogloss:0.15466	test-mlogloss:0.15605
[40]	train-mlogloss:0.08299	test-mlogloss:0.08436
[50]	train-mlogloss:0.04813	test-mlogloss:0.04947
[60]	train-mlogloss:0.03017	test-mlogloss:0.03148
[70]	train-mlogloss:0.02024	test-mlogloss:0.02153
[80]	train-mlogloss:0.01436	test-mlogloss:0.01563
[90]	train-mlogloss:0.01066	test-mlogloss:0.01194
[100]	train-mlogloss:0.00825	test-mlogloss:0.00952
[110]	train-mlogloss:0.00659	test-mlogloss:0.00787
[120]	train-mlogloss:0.00539	test-mlogloss:0.00669
[130]	train-mlogloss:0.00453	test-mlogloss:0.00586
[140]	train-mlogloss:0.00387	test-mlogloss:0.00523
[150]	train-mlogloss:0.00336	test-mlogloss:0.00475
[160]	train-mlogloss:0.00295	test-mlogloss:0.00437
[170]	train-mlogloss:0.00260	test-mlogloss:0.00406
[180]	train-mlogloss:0.00233	test-mlogloss:0.00383
[190]	train-mlogloss:0.00209	test-mlogloss

In [35]:
y_pred_proba = bst.predict(dtest)
y_pred = np.argmax(y_pred_proba, axis=1)

accuracy = accuracy_score(y_test_bert, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9990


In [27]:

sample_resume = "AWS, CI/CD"
# sample_resume = "Python, Java, C, C++, HTML, CSS, JavaScript, SQL, NoSQL, VS Code, Git, GitHub, Figma, WSL,: Linux, Node.js, Express.js, JWT, Passport.js, React.js, Vite, Socket.io , WebRTC, PDFKit,Mongoose, Bootstrap, REST API, EJS, Postman, Docker, AWS Lambda,MySQL, MongoDB, DynamoDB, AWS, Render, NGINX, Apache"
sample_vec = model.encode([sample_resume], convert_to_tensor=False)
dvec = xgb.DMatrix(sample_vec)

probas = bst.predict(dvec)[0]  # shape = (num_classes,)
classes = le.classes_  # your original LabelEncoder classes
job_probs = dict(zip(classes, probas))
target_job = "Full Stack Developer"
chance = job_probs.get(target_job, 0) * 100
print(f"Chance of {target_job}: {chance:.2f}%")
top3 = sorted(job_probs.items(), key=lambda x: x[1], reverse=True)[:3]

print("Top 3 job matches:")
for job, prob in top3:
    print(f"{job}: {prob*100:.2f}%")


Chance of Full Stack Developer: 0.22%
Top 3 job matches:
Cloud Engineer: 85.37%
Security Engineer: 7.89%
DevOps Engineer: 4.65%


In [16]:
import pickle

In [17]:
with open("xgboost_bert_model.pkl", "wb") as f:
    pickle.dump(bst, f)

In [20]:
import joblib
joblib.dump(model, "bert_model_for_xgboost.pkl")

['bert_model_for_xgboost.pkl']

In [21]:
joblib.dump(le, "le_model_for_xgboost.pkl")

['le_model_for_xgboost.pkl']

In [37]:
import numpy as np

In [ ]:
np.save("X_embeddings.npy", X_embeddings)
np.save("y_labels.npy", df_bert['JOB_CATEGORY'].values)